### Przeciążanie operatorów - operatory unarne

Prosta klasa `Vector` z rozdziału 12, która posłuży do testów przeciążania operatorów.

In [100]:
import math
import reprlib
from array import array


class Vector:
    typecode = "d"

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __len__(self):
        return len(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find("[") : -1]
        return "Vector({})".format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

### Przeciążąnie operatora $+$

Przeciążając operatory infiksowe lub unarne, zawsze należy trzymać się fundamentalnej zasady - podczas przeciążania operatorów zawsze powinien być zwracany nowy obiekt danej klasy.

In [13]:
import itertools


class VectorAdd(Vector):
    def __add__(self, other):
        # Jeśli mamy wektory o różnych długościach, to itertools.zip_longest
        # wypełni brakujące wartości zerami.
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return VectorAdd(a + b for a, b in pairs)

In [14]:
v1 = VectorAdd([1, 2, 3])
v2 = VectorAdd([1, 2, 3, 4])
v1 + v2

Vector([2.0, 4.0, 6.0, 4.0])

Metody specjalne implementujące operatory unarne lub infiksowe nigdy nie powinny zmieniać swoich argumentów. Zakłada się, że wyrażenia z takimi operatorami zwracają wyniki tworzące nowe obiekty. Jedynie operatory rozszerzonego przypisania mogą zmieniać argument <code>self</code>.

In [15]:
(1, 2, 3) + v1  # Błąd.

TypeError: can only concatenate tuple (not "VectorAdd") to tuple

Do obsługi operacji dotyczących obiektów różnych typów, Python implementuje specjalny mechanizm dystrybucyjny dla metod specjalnych operatorów infiksowych. Mając wyrażenie $a + b$, interpreter przeprowadzi następujące kroki:

 - Jeśli $a$ ma metodę `__add__`, wywołaj `a.__add__(b)` i zwróć wynik chyba, że jest `NotImplemented`.
 - Jeśli $a$ nie ma metody `__add__` lub jej wywołanie zwraca `NotImplemented`, sprawdź czy $b$ ma metodę `__radd__`, wywołaj `b.__radd__(a)` i zwróć wynik chyba, że jest `NotImplemented`.
 - Jeśli $b$ nie ma metody `__radd__` lub jej wywołanie zwraca `NotImplemented`, zgłoś wyjątek `TypeError`.
  
Metoda `__radd__` jest nazywana "odbitą" lub "odwróconą" wersją metody `__add__`.

Nie należy mylić `NotImplemented` z `NotImplementedError`. `NotImplemented` jest specjalną wartością singletonową, którą metoda specjalna operatora infiksowego powinna zwracać, żeby poinformować interpreter, że nie może obsłużyć danego składnika. Z kolei `NotImplementedError` jest wyjątkiem zgłaszanym przez metody w klasach abstrakcyjnych, aby poinformować, że muszą być one nadpisane w klasach potomnych.

In [16]:
class VectorAdd(Vector):
    def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return VectorAdd(a + b for a, b in pairs)
    
    def __radd__(self, other):
        return self + other

In [17]:
v1 = VectorAdd([1, 2, 3])
(1, 2, 3) + v1

Vector([2.0, 4.0, 6.0])

In [18]:
v1 + "ABC"  # Błąd.

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [19]:
class VectorAdd(Vector):
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return VectorAdd(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

In [20]:
v1 = VectorAdd([1, 2, 3])
v1 + "ABC"  # Teraz poprawny błąd.

TypeError: unsupported operand type(s) for +: 'VectorAdd' and 'str'

### Przeciążanie operatora $*$

In [21]:
class VectorMul(Vector):
    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except ValueError:
            return NotImplemented
        return VectorMul(x * factor for x in self)

    def __rmul__(self, scalar):
        return self * scalar

In [22]:
v1 = VectorMul([1.0, 2.0, 3.0])
v1 * 10

Vector([10.0, 20.0, 30.0])

In [23]:
v1 * False

Vector([0.0, 0.0, 0.0])

In [24]:
from fractions import Fraction

v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

### Przeciążanie operatora $@$

In [74]:
from collections import abc


class VectorMatMul(Vector):
    def __matmul__(self, other):
        # Obydwa operandy muszą implementować __len__ oraz __iter__
        if not (isinstance(other, abc.Sized) and isinstance(other, abc.Iterable)):
            return NotImplemented
        try:
            return sum(a * b for a, b in zip(self, other, strict=True))
        except ValueError:
            raise ValueError("@ requires vectors of equal length")

    def __rmatmul__(self, other):
        return self @ other

In [75]:
v1 = VectorMatMul([1, 2, 3])
v1 @ [1, 2, 3]

14.0

In [76]:
v2 = VectorMatMul([3, 4, 5])
v2 @ v1

26.0

### Poprawiona metoda `__eq__`

In [102]:
class VectorEq(Vector):
    def __eq__(self, other):
        if not isinstance(other, VectorEq):
            return NotImplemented
        try:
            return all(a == b for a, b in zip(self, other, strict=True))
        except ValueError:
            raise ValueError("== requires vectors of equal length")

In [103]:
v1 = VectorEq([1, 2, 3])
v2 = VectorEq([1, 2, 3])
v1 == v2

True

In [104]:
v1 == (1, 2, 3)

False

### Operatory rozszerzonego przypisania

Jeżeli klasa nie implementuje operatorów działających w miejscu, to operator rozszerzonego przypisania działa jako lukier syntaktyczny: $a += b$ jest wykonywane jako $a = a + b$. Jest to oczekiwane zachowanie dla typów niezmiennych. Jeśli mamy zdefiniowaną metodę `__add__` to operator $+=$ będzie działał bez dodatkowego kodu. Jednakże jeśli zaimplementujemy metodę w miejscu `__iadd__` to będzie ona modyfikowała lewy operand a nie tworzyła nowy obiekt. 

Metody specjalne "w miejscu" nigdy nie powinny być implementowane dla typów niezmiennych, np. takich jak klasa `Vector`.